In [1]:
import requests
import pandas as pd
import numpy as np
from pybtex.database import BibliographyData, Entry
import re

In [11]:
class NOAAStudies:
    def __init__(self):
        """
        Initialize the NOAAStudies class with base URL and dictionaries to hold studies and data table indices.


        The search parameters are designed to be flexible and can vary for different searches using the same object.
        Instead of overwriting the initialized default parameters, each search maintains its own set of parameters, 
        allowing for independent configurations across multiple searches.
        """
        self.BASE_URL = "https://www.ncei.noaa.gov/access/paleo-search/study/search.json"
        self.studies = {}
        self.data_table_index = {}

    def search_studies(self, xml_id=None, noaa_id=None, data_publisher="NOAA", data_type_id = None, keywords = None, investigators=None, 
                       max_lat=None, min_lat=None, max_lon=None, min_lon=None, location = None, publication = None, search_text = None, 
                       earliest_year = None, latest_year = None, cv_whats = None, recent = False):
        """
        Search for NOAA studies using specific search parameters.
        
        Parameters:
            xml_id (str): XML ID of the study. (Primary)
            noaa_id (str): NOAA study ID. (Primary)
            data_publisher (str): Data publisher's name. Default: NOAA 
            data_type_id (str): Data Type specific Studies.
            keywords (str): Keywords to search for.  
            investigators (str): Name(s) of investigators.
            min_lat/max_lat (float): Latitude range for location-based search.
            min_lat/max_lat (float): Longitude range for location-based search.
            location (str): Location description.
            species (str): FOUR letter code species code
            publication (str): Specific publication.
            search_text (str): Publication / studyNotes / String based search.
           
        """
        if noaa_id:
            params = {'NOAAStudyId': noaa_id}
        elif xml_id:
            params = {'xmlId': xml_id}
        else:
            params = {
            'dataPublisher' : data_publisher,
            'dataTypeId' : data_type_id,
            'keywords' : keywords, 
            'investigators' : investigators,
            'minLat' : min_lat,
            'maxLat' : max_lat,
            'minLon' : min_lon,
            'maxLon' : max_lon,
            'locations' : location,
            'searchText' : publication, 
            'searchText' : search_text,
            'cvWhats': cv_whats,
            'earliestYear' : earliest_year,
            'latestYear': latest_year, 
            'recent' : recent,
        }
        # Filtering out None values
        params = {k: v for k, v in params.items() if v}
        response = requests.get(self.BASE_URL, params=params)
        if response.status_code == 200:
            self.response_parser(response.json())
        else:
            print(f"Error fetching studies: {response.status_code}")

    def params_validators(self, key, value):
        """
        - Validations: 
            - minLon, maxLon, minLat, maxLat:
                Latitude Shall be in the -90 to +90 range
                Longitude shall be in the -180 to +180 range
            - Parse multiple inputs: 
                Implement parsing for more than one keyword, DataTypeId
            - species:
                Limit till 4 letter code
            - earliestYear/ latestYear:
            Validate Year

        """
        pass

    def response_parser(self, data):
        """
        Parse the JSON response from NOAA and populate the studies dictionary.
        
        Parameters:
            data (dict): The JSON data returned from a search query.
        """
        for study in data.get('study', []):
            noaa_study_id = study.get('NOAAStudyId')
            xml_study_id = study.get('xmlId')
            self.studies[noaa_study_id] = {
                'base_meta': self.load_base_meta(study),
                'investigators': self.load_investigators(study),
                'publications': self.load_publications(study),
                'sites': self.load_sites(study, noaa_study_id),
                # 'number of sites': len(sites)
                'pageUrl' : study.get('onlineResourceLink', np.nan)
            }

    def load_base_meta(self, study):
        """
        Load base metadata for a study.
        
        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
        """
        fields = ['NOAAStudyId', 'studyName', 'dataType', 'earliestYearBP', 'mostRecentYearBP',
                  'earliestYearCE', 'mostRecentYearCE', 'studyNotes', 'scienceKeywords']
        return {field: study.get(field, np.nan) for field in fields}

    def load_investigators(self, study):
        """
        Extract investigator details from the study data.
        
        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
        """
        investigators = study.get("investigatorDetails", [])
        if investigators:
            return ", ".join([f"{i.get('firstName', 'N/A')} {i.get('lastName', 'N/A')}" for i in investigators])
        return np.nan

    def load_publications(self, study):
        """
        Extract and format publication data from the study as a dictionary.

        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
            study_id (str): The unique identifier for the NOAA study.

        Returns:
            list: A list of dictionaries representing publication details.
        """
        publications = []
        for pub in study.get('publication', []):
            author_info = pub.get('author', {})
            identifier_info = pub.get('identifier', {})

            # Extract fields for the publication dictionary
            publication_data = {
                'NOAAStudyId': study.get('NOAAStudyId'),
                'author': author_info.get('name', 'Unknown Author'),
                'title': pub.get('title', 'Unknown Title'),
                'journal': pub.get('journal', 'Unknown Journal'),
                'year': str(pub.get('pubYear', 'Unknown Year')),
                'volume': pub.get('volume', np.nan),
                'number': pub.get('issue', np.nan),
                'pages': pub.get('pages', np.nan),
                'type': pub.get('type', np.nan),
                'doi': identifier_info.get('id', np.nan) if identifier_info else np.nan,
                'url': identifier_info.get('url', np.nan) if identifier_info else np.nan
            }

            # Add the publication dictionary to the list
            publications.append(publication_data)
        
        return publications

    def load_sites(self, study, study_id):
        """
        Load and format site data associated with the study.
        
        Parameters:
            study (dict): Part of the JSON data pertaining to a single study.
            study_id (str): The unique identifier of the study for reference.
        """        
        return {
            site.get('NOAASiteId', np.nan): {
                'siteName': site.get('siteName', np.nan),
                'locationName': site.get('locationName', np.nan),
                'lat': site.get('geo', {}).get('geometry', {}).get('coordinates', [None, None])[0],
                'lon': site.get('geo', {}).get('geometry', {}).get('coordinates', [None, None])[1],
                'paleoData': self.load_paleo_data(site.get('paleoData', []), study_id, site.get('NOAASiteId'))
            }
            for site in study.get('site', [])
        }

    def load_paleo_data(self, paleoData, study_id, site_id):
        """
        Extract and format paleo data associated with a site.
        
        Parameters:
            paleoData (list): List of paleo data from the site.
            study_id (str): The unique identifier of the study.
            site_id (str): The unique identifier of the site.
        """
        paleo_dict = {}
        for paleo in paleoData:
            # Safe access to 'dataFile' list
            data_files = paleo.get('dataFile', [])
            file_url = data_files[0].get('fileUrl', np.nan) if data_files else np.nan
            variables = []
            if data_files:  # Check if 'dataFile' is not empty
                variables = [var.get('cvShortName', np.nan) for var in data_files[0].get('variables', [])]
            data_table_id = paleo.get('NOAADataTableId', np.nan)
            paleo_details = {
                'NOAADataTableId': data_table_id,
                'dataTableName': paleo.get('dataTableName', np.nan),
                'timeUnit': paleo.get('timeUnit', np.nan),
                'fileUrl': file_url,
                'variables': variables
            }
            paleo_dict[data_table_id] = paleo_details

            self.data_table_index[data_table_id] = {
                'file_url': file_url,
                'study_id': study_id,
                'site_id': site_id
            }
        return paleo_dict

    def assert_list(self, input_item):
        """
        Ensure the input is a list. If it's not a list, convert it into a single-element list.

        Parameters:
            input_item: A single element or a list.

        Returns:
            list: A list containing the input item(s).
        """
        if isinstance(input_item, list):
            return input_item
        elif input_item is not None:
            return [input_item]
        else:
            return []
        
    def get_response(self):
        """
        Compile and return a DataFrame of all loaded studies along with their detailed metadata and linked information.
        
        Returns:
            DataFrame: A DataFrame representing the consolidated data of all studies, including metadata, investigators,
                       publications, and site details.
        """
        data = [{
            **study['base_meta'],
            'Investigators': study['investigators'],
            'publications': study['publications'],
            'sites': study['sites']
        } for study in self.studies.values()]
        return pd.DataFrame(data)

    def get_citation(self, publication):
        """
        Generate a citation key for a publication.

        Parameters:
            publication (dict): A dictionary containing publication details.

        Returns:
            str: A unique citation key.
        """
        # Extract last name from the author field
        last_name = publication.get('author', 'Unknown Author').split()[-1]

        # Extract first significant word from the title
        title = publication.get('title', 'Unknown Title')
        words = re.findall(r'\w+', title)
        first_significant_word = next((word.capitalize() for word in words if len(word) > 2 and word.lower() != 'the'), "Unknown")

        # Get the year and NOAAStudyId
        pub_year = publication.get('year', 'Unknown Year')
        study_id = publication.get('NOAAStudyId', 'UnknownID')

        # Create citation key
        return f"{last_name}_{first_significant_word}_{pub_year}_{study_id}".replace(" ", "")
    
    def get_publications(self, study_ids, output_format="dataframe"):
        """
        Return publications for one or more study IDs in the specified format.

        Parameters:
            study_ids (list or str): Single or list of NOAAStudyIds.
            output_format (str): The desired output format. Options:
                - "dataframe": Returns a pandas DataFrame of publication details.
                - "bibtex": Returns a pybtex.database.BibliographyData object.

        Returns:
            pd.DataFrame or pybtex.database.BibliographyData: Publications data in the chosen format.
        """
        study_ids = self.assert_list(study_ids)  # Ensure study_ids is a list

        if output_format == "dataframe":
            dfs = []
            for study_id in study_ids:
                if study_id in self.studies:
                    publications = self.studies[study_id].get('publications', [])
                    df = pd.DataFrame(publications)
                    # df.set_index('NOAAStudyId', inplace=True)  # Set NOAAStudyId as the index
                    dfs.append(df)
                else:
                    print(f"Study ID {study_id} not found.")
            return pd.concat(dfs) if dfs else pd.DataFrame()

        elif output_format == "bibtex":
            from pybtex.database import BibliographyData, Entry
            bib_entries = {}
            for study_id in study_ids:
                if study_id in self.studies:
                    publications = self.studies[study_id].get('publications', [])
                    for pub in publications:
                        fields = {k: v for k, v in pub.items() if k not in ['NOAAStudyId'] and v}
                        entry = Entry('article', fields=fields)
                        citation_key = self.get_citation(pub)
                        bib_entries[citation_key] = entry
                else:
                    print(f"Study ID {study_id} not found.")
            return BibliographyData(entries=bib_entries)

        else:
            print("Invalid output_format. Choose 'dataframe' or 'bibtex'.")

    def get_sites(self, study_ids):
        """
        Return a DataFrame of sites for one or more study IDs.

        Parameters:
            study_ids (list or str): Single or list of NOAAStudyIds.

        Returns:
            pd.DataFrame: Sites DataFrame.
        """
        if not isinstance(study_ids, list):
            study_ids = [study_ids]  # Convert to list if single ID
        
        dfs = []
        for study_id in study_ids:
            if study_id not in self.studies:
                print(f"Study ID {study_id} not found.")
                continue
            
            sites_data = self.studies[study_id].get('sites', {})
            sites_list = []
            for site_id, site_info in sites_data.items():
                site_info['NOAASiteId'] = site_id
                paleo_list = site_info.pop('paleoData', {})
                for paleo_id, paleo_info in paleo_list.items():
                    record = {**site_info, **paleo_info, 'NOAADataTableId': paleo_id, 'NOAAStudyId': study_id}
                    sites_list.append(record)

            if sites_list:
                df = pd.DataFrame(sites_list)
                dfs.append(df)
                # print(dfs)
        
        result = pd.concat(dfs) if dfs else pd.DataFrame()
        result.set_index('NOAAStudyId', inplace=True)  # Set NOAAStudyId as the index
        return result


    def get_data(self, dataTableIDs=None, file_urls=None):
        """
        Fetch and return the data for one or more dataTableIDs or file URLs.

        Parameters:
            dataTableIDs (list or str): Single or list of NOAADataTableIds.
            file_urls (list or str): Single or list of file URLs.

        Returns:
            pd.DataFrame: Combined DataFrame of all fetched data.
        """

        """ 
        @TODO: Add attributes to each data frame 
        """
        if dataTableIDs:
            if not isinstance(dataTableIDs, list):
                dataTableIDs = [dataTableIDs]  # Convert to list if single ID
            dfs = []
            for dataTableID in dataTableIDs:
                file_url = self.data_table_index.get(dataTableID, {}).get('file_url')
                if not file_url:
                    print(f"Data Table ID {dataTableID} not found or no associated file URL.")
                    continue
                dfs.append(self._fetch_data(file_url))
            return dfs
        
        if file_urls:
            if not isinstance(file_urls, list):
                file_urls = [file_urls]  # Convert to list if single URL
            dfs = [self._fetch_data(file_url) for file_url in file_urls]
            return dfs

        print("No dataTableID or file URL provided.")
        return pd.DataFrame()

    def _fetch_data(self, file_url):
        """
        Helper method to fetch data from a file URL and return it as a DataFrame.
        """
        response = requests.get(file_url)
        if response.status_code == 200:
            lines = response.text.split('\n')
            data_lines = [line for line in lines if not line.startswith('#') and line.strip()]
            if data_lines:
                headers = data_lines[0].split('\t')
                data = [line.split('\t') for line in data_lines[1:]]
                return pd.DataFrame(data, columns=headers)
        print(f"Failed to fetch data from {file_url}.")
        return pd.DataFrame()

In [3]:
# Example usage:
studies = NOAAStudies()
studies.search_studies(investigators="Bhattacharya")
display(studies.get_response())

,NOAAStudyId,studyName,dataType,earliestYearBP,mostRecentYearBP,earliestYearCE,mostRecentYearCE,studyNotes,scienceKeywords,Investigators,publications,sites
0,37258,Benguela Upwelling System Hydrogen and Carbon ...,PALEOCEANOGRAPHY,5183867,0,-5181917,1950,Hydrogen Isotopic Reconstruction of North Amer...,[hydrology],"Claire Rubbelke, Tripti Bhattacharya, Ran Feng...","[{'NOAAStudyId': '37258', 'author': 'Rubbelke,...","{'59689': {'siteName': 'ODP 1081', 'locationNa..."
1,36778,California Margin Hydrogen and Carbon Isotope ...,PALEOCEANOGRAPHY,3700000,0,-3698050,1950,Hydrogen Isotopic Reconstruction of North Amer...,"[Monsoon, hydrology]","Tripti Bhattacharya, Ran Feng, Jessica Tierney...","[{'NOAAStudyId': '36778', 'author': 'Bhattacha...","{'30617': {'siteName': 'ODP 1012', 'locationNa..."
2,39200,"Clayton Valley, Nevada Hydrogen and Carbon Iso...",PALEOLIMNOLOGY,2800000,0,-2798050,1950,Hydrogen and carbon isotopes of long-chain lea...,None,"Tripti Bhattacharya, Peter Brennan, Daniel Iba...","[{'NOAAStudyId': '39200', 'author': 'Gagnon, C...","{'60188': {'siteName': 'Clayton Valley', 'loca..."
3,38799,Eastern Equatorial Pacific Leaf Wax Isotope Da...,PALEOCEANOGRAPHY,4956,517,-3006,1433,Age assigned based on age models from Etournea...,None,"David Fastovich, Tripti Bhattacharya, Lina Pér...","[{'NOAAStudyId': '38799', 'author': 'Fastovich...","{'56876': {'siteName': 'ODP 1239', 'locationNa..."
4,36293,Eastern Pacific Alkenone Sea Surface Temperatu...,CLIMATE RECONSTRUCTIONS,144467,2430,-142517,-480,None,None,"Dervla Meegan Kumar, Jessica Tierney, Tripti B...","[{'NOAAStudyId': '36293', 'author': 'Meegan Ku...","{'19117': {'siteName': 'NH22P', 'locationName'..."
5,37679,Geochemical Proxies from a Northeast Mexico Sp...,SPELEOTHEMS,62500,5100,-60550,-3150,None,"[Milankovitch, Precipitation Reconstruction, L...","David McGee, Gabriela Serrato Marks, Kathleen ...","[{'NOAAStudyId': '37679', 'author': 'Kevin T. ...","{'59509': {'siteName': 'Cueva Bonita', 'locati..."
6,24890,Guaymas Basin 9-12kYrBP Leaf Wax Isotopes and ...,PALEOCEANOGRAPHY,24266,1460,-22316,490,Stable isotope (dD and d13C) data on leaf waxe...,[Monsoon],"Tripti Bhattacharya, Jessica Tierney, Jason Ad...","[{'NOAAStudyId': '24890', 'author': 'Tripti Bh...","{'53312': {'siteName': 'MD02-2515', 'locationN..."
7,17735,"Laguna de Aljojuca, Mexico 4000 Year Stable Is...",PALEOLIMNOLOGY,4139,-65,-2189,2015,Fluctuations in climate over the past 4000 cal...,[drought],"Tripti Bhattacharya, None Byrne, Harald Böhnel...","[{'NOAAStudyId': '17735', 'author': 'Tripti Bh...","{'56460': {'siteName': 'Laguna de Aljojuca', '..."
8,36793,Leaf Wax Hydrogen Isotope Data from the Chilea...,PALEOCEANOGRAPHY,24225,-26,-22275,1976,None,"[westerlies, Last Glacial Maximum]","Aria Blumm, Jessica Tierney, Tripti Bhattachar...","[{'NOAAStudyId': '36793', 'author': 'Blumm, Ar...","{'19251': {'siteName': 'ODP 1233', 'locationNa..."
9,33012,Mesoamerica Last Millennium Lake and Speleothe...,CLIMATE RECONSTRUCTIONS,1100,100,850,1850,Synthesis of lacustrine records of hydroclimat...,[Other Hydroclimate Reconstruction],"Tripti Bhattacharya, Sloan Coats","[{'NOAAStudyId': '33012', 'author': 'Tripti Bh...","{'58864': {'siteName': 'Mesoamerica', 'locatio..."


In [4]:
#  Display multiple studies' multiple publication, with each row being identified by a NOAAStudyId
display(studies.get_publications(['36778', '37258', '17735']))

,NOAAStudyId,author,title,journal,year,volume,number,pages,type,doi,url
0,36778,"Bhattacharya, Tripti; Feng, Ran; Tierney, Jess...",Expansion and Intensification of the North Ame...,AGU Advances,2022,3,6,None,publication,10.1029/2022AV000757,http://dx.doi.org/10.1029/2022AV000757
0,37258,"Rubbelke, Claire; Bhattacharya, Tripti; Feng, ...",Plio-Pleistocene Southwest African Hydroclimat...,Geophysical Research Letters,2023,50,None,None,publication,10.1029/2023GL103003,http://dx.doi.org/10.1029/2023GL103003
0,17735,"Tripti Bhattacharya, Roger Byrne, Harald Bohne...",Cultural implications of late Holocene climate...,Proceedings of the National Academy of Sciences,2015,None,None,None,publication,10.1073/pnas.1405653112,http://dx.doi.org/10.1073/pnas.1405653112


In [5]:
#  Display multiple studies' multiple sites, with each row being identified by a NOAAStudyId
display(studies.get_sites(['36778', '37258', '17735']))

,siteName,locationName,lat,lon,NOAASiteId,NOAADataTableId,dataTableName,timeUnit,fileUrl,variables
NOAAStudyId,,,,,,,,,,
36778,ODP 1012,Ocean>Pacific Ocean>North Pacific Ocean,32.28,-118.38,30617,49503,ODP1012-leafwax Bhattacharya2022,cal yr BP,https://www.ncei.noaa.gov/pub/data/paleo/paleo...,"[age_ma, dD_wax, dD_wax_err, d13C_wax, d13C_wa..."
36778,DSDP 475,Ocean>Pacific Ocean>Eastern Pacific Ocean,23.03,-109.03,59597,49502,DSDP475-leafwax Bhattacharya2022,cal yr BP,https://www.ncei.noaa.gov/pub/data/paleo/paleo...,"[age_ma, dD_wax, dD_wax_err, d13C_wax, d13C_wa..."
37258,ODP 1081,Ocean>Atlantic Ocean>South Atlantic Ocean,-19.616667,11.316667,59689,49940,ODP1081-leafwax isotopes Rubbelke2023,cal yr BP,https://www.ncei.noaa.gov/pub/data/paleo/paleo...,"[age_ma, depth, C31_dD, d13C_wax, d13C_stdev, ..."
17735,Laguna de Aljojuca,Continent>North America>Mexico,19.0915,-97.5338,56460,28054,Aljojuca2015iso,cal yr BP,https://www.ncei.noaa.gov/pub/data/paleo/paleo...,"[None, None, None, None, None, None]"
17735,Laguna de Aljojuca,Continent>North America>Mexico,19.0915,-97.5338,56460,28055,Aljojuca2015XRF,cal yr BP,https://www.ncei.noaa.gov/pub/data/paleo/paleo...,"[None, None, None, None, None, None, None, Non..."


In [6]:
data = studies.get_data(['49502','49940', '28055'])

In [7]:
data[0]

,age_ma,dD_wax,dD_wax_err,d13C_wax,d13C_wax_err,dD_precip,dD_precip_err,\r
0,0.002,-147,2,-30.8,0.2,-63,6,\r
1,0.003,-144,2,NaN,NaN,-60,6,\r
2,0.01143695,-145,2,-30.5,0.2,-62,6,\r
3,0.215249267,-146,2,-29.0,0.2,-61,6,\r
4,0.265102639,-145,2,-29.0,0.2,-64,6,\r
...,...,...,...,...,...,...,...,...
93,3.327702703,-130,2,NaN,NaN,-44,6,\r
94,3.441216216,-127,2,-30.4,NaN,-40,6,\r
95,3.672972973,-133,2,-30.9,0.2,-49,5,\r
96,3.725,-135,2,-29.9,0.2,-50,6,\r


In [8]:
data[1]

,age_ma,depth,C31_dD,C31_dD_stdev,d13C_wax,d13C_stdev,dD_precip_estimates,dD_precip_err,epsilon,acl,\r
0,0,3,-162.7238,1.3507,NaN,NaN,-16.31597,4.972454,-119.691,NaN,\r
1,1.454545455,16,-157.3797,2.243924,-24.46768,0.20191,-46.92634,3.784594,-119.691,27.51592,\r
2,4.157104558,49,-149.0226,3.145416,-25.83589,0.083577,-42.75867,3.522674,-120.221,30.19233,\r
3,25.32868633,314,-151.5405,1.163235,-22.90912,0.041797,-35.59529,4.165406,-120.516,28.18216,\r
4,166.9043133,1406,-165.4232,1.876341,-25.0265,0.035033,-38.92805,4.233686,-119.429,28.67139,\r
...,...,...,...,...,...,...,...,...,...,...,...
80,4765.467177,28341,-123.4744,0.65349,-25.75246,0.073276,-40.04986,3.614472,-119.097,27.04846,\r
81,4843.526966,28641,-130.5614,1.694729,-26.71182,0.129743,-3.075839,3.6006,-118.553,NaN,\r
82,4978.310201,29159,-129.2111,1.555938,-25.70897,0.135756,-12.41682,3.349344,-119.105,30.25767,\r
83,5017.340095,29309,-142.0427,0.225141,-26.01416,0.173679,-10.05254,3.595759,-118.901,30.22954,\r


In [9]:
data[2]

,depth_cm,age_calBP95-,age_calBP95+,age_calBP,CaO%-x,Al2O3%-x,Ti,Zr,PC1\r
0,10,-282.7,15.2,-95.9,9.198,13.275,4086,139.5,0.16858631\r
1,20,-269.7,31.5,-80.2,10.182,13.644,4190,147.2,0.188918422\r
2,30,-251.9,52.4,-64.6,8.653,12.864,3911,141.4,0.18122268\r
3,60,-213.7,117,-18.7,10.253,13.385,4150,146.7,0.344839954\r
4,80,-170.9,145.8,12.6,9.945,13.656,4305,160.6,0.361033968\r
...,...,...,...,...,...,...,...,...,...
113,820,2488.4,3331.2,2867.4,11.701,11.621,3721,136.6,0.022378136\r
114,824,2499.7,3404.9,2906.3,10.831,11.682,3747,138.3,-0.766964569\r
115,830,2513.4,3541.6,2964.7,12.426,11.995,3520,132.3,0.882225526\r
116,835,2525.6,3656.7,3013.5,10.508,10.623,3670,142.1,-0.792353741\r
